In [17]:
import time
from pathlib import Path

import pandas as pd
from selenium.webdriver.common.by import By

from config import Config
from divida_ativa_page import get_divida_ativa_driver, get_element_by_xpath

### File Locations

In [18]:
in_file_estoque = Path.cwd() / "data" / "raw" / "2024-01-11 Lista estoque dívida acima de 50000.xlsx"
in_file_parcelas = Path.cwd() / "data" / "raw" / "2024-01-11 Lista de parcelas acima de 500 a partir de 01-11-2023.xlsx"
in_file_memo = Path.cwd() / "data" / "raw" / "memo.csv"
summary_file_imobiliario = Path.cwd() / "data" / "processed" / f"dados_cadastrais_mobiliario.csv"
summary_file_mobiliario = Path.cwd() / "data" / "processed" / f"dados_cadastrais_imobiliario.csv"

In [19]:
dtypes={
    'IdCadastro': 'string'
}

df_estoque = pd.read_excel(in_file_estoque, dtype=dtypes)
df_parcelas = pd.read_excel(in_file_parcelas, dtype=dtypes)


### Column Cleanup

- Remove all leading and trailing spaces
- Rename the columns for consistency.

In [20]:
# https://stackoverflow.com/questions/30763351/removing-space-in-dataframe-python
df_estoque.columns = [x.strip() for x in df_estoque.columns]
df_parcelas.columns = [x.strip() for x in df_parcelas.columns]

In [21]:
{col: '' for col in df_estoque.columns}

{'Nome da Origem': '',
 'Tipo': '',
 'Cadastro': '',
 'CD': '',
 'Exercício': '',
 'Situação': '',
 'Origem': '',
 'Lancado': '',
 'Multa': '',
 'Juros': '',
 'Correção': '',
 'Subtotal': '',
 'Lançamento': '',
 'Inscrição': '',
 'Livro': '',
 'Folha': '',
 'Ano_lancamento': '',
 'Ano_inscricao': '',
 'IdCadastro': '',
 'Situacao_Tratada': '',
 'Exercicio_Tratado': '',
 'Data da Origem': '',
 'tamanho_divida': ''}

In [22]:
{col: '' for col in df_parcelas.columns}

{'IdAcordo': '',
 'NumParcela': '',
 'DataAcordo': '',
 'Vencimento': '',
 'IdParcela': '',
 'ValorParcela': '',
 'mes': '',
 'nome_mes': '',
 'trimes': '',
 'ano': '',
 'Situacao Pagamento': '',
 'IdCadastro': '',
 'Status_revi': '',
 'ValorPago': '',
 'ValorParcelaTratado': '',
 'Situacao Acordo': ''}

### Clean Up Data Types

In [ ]:
print(df_estoque.dtypes)
print(df_parcelas.dtypes)

### Data Manipulation

In [ ]:
cols_to_rename = {'col1': 'New_Name'}
df_estoque.rename(columns=cols_to_rename, inplace=True)
df_parcelas.rename(columns=cols_to_rename, inplace=True)

In [25]:
driver = get_divida_ativa_driver()



In [ ]:
driver.get(Config.URL_IMOB)

In [ ]:
campo_inscricao = get_element_by_xpath(driver, '//input[@name="ic"]')
campo_inscricao.clear()
campo_inscricao.send_keys('70945')


In [ ]:
campo_digito = get_element_by_xpath(driver, '//input[@name="digito"]')
campo_digito.clear()
campo_digito.send_keys('0')

In [ ]:
campo_documento = get_element_by_xpath(driver, '//input[@name="docto"]')
campo_documento.clear()
campo_documento.send_keys('0')

In [ ]:
btn_continuar = get_element_by_xpath(driver, '//input[@type="image"]')
btn_continuar.click()

In [ ]:
btn_consultar_dados = get_element_by_xpath(driver, '//img[@src="/ps_image/btw.php?cont=Consultar D.A."]')
btn_consultar_dados.click()

In [ ]:
btn_dados_cadastrais = get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]')
btn_dados_cadastrais.click()

In [ ]:
def get_dados_cadastrais_imobiliario(driver, tipo, inscricao):
    url_imobiliario = Config.URL_IMOB
    
    
    for _ in range(2):
        driver.get(url_imobiliario)
        
        campo_inscricao = get_element_by_xpath(driver, '//input[@name="ic"]')
        campo_inscricao.clear()
        campo_inscricao.send_keys(inscricao)

        campo_digito = get_element_by_xpath(driver, '//input[@name="digito"]')
        campo_digito.clear()
        campo_digito.send_keys('0')

        campo_documento = get_element_by_xpath(driver, '//input[@name="docto"]')
        campo_documento.clear()
        campo_documento.send_keys('0')

        btn_continuar = get_element_by_xpath(driver, '//input[@type="image"]')
        btn_continuar.click()

        btn_consultar_dados = get_element_by_xpath(driver, '//img[@src="/ps_image/btw.php?cont=Consultar D.A."]', max_wait=3)
        if btn_consultar_dados is None:
            continue            
        btn_consultar_dados.click()

        btn_dados_cadastrais = get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]')
        
        if btn_dados_cadastrais is not None:
            btn_dados_cadastrais.click()
            break

        inscricao = int(inscricao)
        
    else:
        return None

    dado_inscricao = None
    dado_responsavel = None
    dado_endereco_completo = None
    dado_cpf_cnpj = None
    dado_local_imovel = None
    dado_endereco_correio = None

    get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//tr//')
    
    table_header = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//td')
    dado_inscricao, dado_responsavel = [d.text for d in table_header][0].split(' - ')

    table_body = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//following::table[1]//tr')
    for i, row in enumerate(table_body):
        dados = row.find_elements(By.TAG_NAME, 'td')
        if i == 0:
            _, dado_endereco_completo, _, dado_cpf_cnpj = [d.text for d in dados]
        if i == 1:
            _, dado_local_imovel = [d.text for d in dados]
            
        if i == 2:
            dado_local_imovel += f' - {row.text}'

        if i == 3:
            _, dado_endereco_correio = [d.text for d in dados]

        if i == 4:
            dado_endereco_correio += f' - {row.text}'

    
    return dado_inscricao, dado_responsavel, dado_endereco_completo, dado_cpf_cnpj, dado_local_imovel, dado_endereco_correio

In [ ]:
dado_inscricao = None
dado_responsavel = None 
get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//tr//')
table_header = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//td')
dado_inscricao, dado_responsavel = [d.text for d in table_header][0].split(' - ')

print(dado_inscricao,':', dado_responsavel)

In [ ]:
get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//tr//')
table_body = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//following::table[1]//tr')

dado_endereco_completo = None
dado_cpf_cnpj = None
dado_local_imovel = None
dado_endereco_correio = None

for i, row in enumerate(table_body):
    dados = row.find_elements(By.TAG_NAME, 'td')
    if i == 0:
        _, dado_endereco_completo, _, dado_cpf_cnpj = [d.text for d in dados]
    if i == 1:
        _, dado_local_imovel = [d.text for d in dados]
        
    if i == 2:
        dado_local_imovel += f' - {row.text}'

    if i == 3:
        _, dado_endereco_correio = [d.text for d in dados]

    if i == 4:
        dado_endereco_correio += f' - {row.text}'

        
        print(dado_endereco_correio)
# print('dado_cpf_cnpj:', dado_cpf_cnpj)
# print('dado_endereco_completo:', dado_endereco_completo)


In [ ]:
df_imobiliario = pd.DataFrame(columns=['inscricao', 'responsavel', 'endereco_completo', 
                                      'cpf_cnpj', 'local_imovel', 'endereco_correio'])

df_mobiliario = pd.DataFrame(columns=['inscricao', 'razao_social', 'endereco', 'documento', 
                                      'nome_fantasia', 'estabelecimento', 'tipo_tributo', 
                                      'nome_socio', 'cpf_socio', 'rg_socio', 'data_socio'])

for cadastro in df_parcelas['IdCadastro']:
    tipo, inscricao = cadastro[:-11].replace('0', ''), cadastro[-11:]

    if tipo == '1':
        dados_cadastrais = get_dados_cadastrais_imobiliario(driver, tipo, (inscricao))
        if dados_cadastrais is None:
            continue
        df_imobiliario.loc[len(df_imobiliario)] = dados_cadastrais

    if tipo == '2':
        inscricao = inscricao[-5:]
        dados_cadastrais_list = get_dados_cadastrais_mobiliario(driver, tipo, inscricao)
        if dados_cadastrais is None:
            continue
        for dados_cadastrais in dados_cadastrais_list:
            df_mobiliario.loc[len(df_mobiliario)] = dados_cadastrais



for cadastro in df_estoque['IdCadastro']:
    tipo, inscricao = cadastro[:-11].replace('0', ''), cadastro[-11:]
    if tipo == '2':
        inscricao = inscricao[-5:]
    print(tipo, inscricao)

df_imobiliario

In [ ]:
df_imobiliario

In [37]:
driver.get(Config.URL_MOB)

In [39]:
campo_cadastro = get_element_by_xpath(driver, '//input[@name="cadm"]')
campo_cadastro.clear()
# campo_cadastro.send_keys('12902')
campo_cadastro.send_keys('70945')

In [40]:
campo_digito = get_element_by_xpath(driver, '//input[@name="digito"]')
campo_digito.clear()
campo_digito.send_keys('0')

In [41]:
campo_documento = get_element_by_xpath(driver, '//input[@name="docto"]')
campo_documento.clear()
campo_documento.send_keys('0')

In [42]:
btn_continuar = get_element_by_xpath(driver, '//input[@type="image"]')
btn_continuar.click()

In [43]:
btn_consultar_dados = get_element_by_xpath(driver, '//img[@src="/ps_image/btw.php?cont=Consultar D.A."]')
btn_consultar_dados.click()

In [44]:
btn_dados_cadastrais = get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]')
btn_dados_cadastrais.click()

In [45]:
get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//tr//')

dado_inscricao = None
dado_razao_social = None
table_header = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//td')
dado_inscricao, dado_razao_social = [d.text.strip() for d in table_header][0].split(' - ')

In [46]:
dado_endereco = None
dado_documento = None
dado_nome_fantasia = None
dado_estabelecimento = None
dado_tipo_tributo = None

table_body = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//following::table[1]//tr')
dado_endereco, dado_documento = ' '.join([d.text for d in table_body][0].split()[:-2]), [d.text for d in table_body][0].split()[-1]
dado_endereco += f' - {[d.text for d in table_body][1]}'
dado_nome_fantasia = [d.text for d in table_body][2].split(':')[-1]
dado_estabelecimento = [d.text for d in table_body][3].split(':')[-1]
dado_tipo_tributo = [d.text for d in table_body][4].split(':')[-1]

dado_endereco, dado_documento, dado_nome_fantasia, dado_estabelecimento, dado_tipo_tributo

('Endereço: AV LOURENCO DE SOUZA FRANCO - MOGI DAS CRUZES SP 08750-560',
 '16.949.657/0001-30',
 '',
 ' 4321500 - INSTALACAO E MANUTENCAO ELETRI',
 ' 3 - Tributo nao cadastrado')

In [35]:
table_body = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//following::table[2]//tr')
socio_list = []
for i, row in enumerate(table_body):
    dados = row.find_elements(By.TAG_NAME, 'td')
    if i == 0:
        if 'C.P.F' not in row.text.upper():
            socio = [None, None, None, None]
            socio_list.append(socio)
            break
        continue
    socio = [d.text for d in dados]
    socio_list.append(socio)

for s in socio_list:
    print(s)

['1999', 'Tx.Licenca', '35', '000', '', '005.212/2003', 'Pago', '']
['2000', 'Tx.Licenca', '35', '000', '', '005.212/2003', 'Pago', '']
['2001', 'Tx.Licenca', '35', '000', '', '005.212/2003', 'Pago', '']
['2002', 'Tx.Licenca', '35', '000', '', '005.212/2003', 'Parcelamento:027.121/2023', '']
['2003', 'Tx.Licenca', '35', '000', '', '005.770/2007', 'Parcelamento:027.121/2023', '']
['2004', 'Tx.Licenca', '35', '000', '', '005.770/2007', 'Parcelamento:027.121/2023', '']
['2005', 'Tx.Licenca', '35', '000', '', '005.770/2007', 'Parcelamento:027.121/2023', '']
['2006', 'Tx.Licenca', '35', '000', '', '005.770/2007', 'Parcelamento:027.121/2023', '']
['2007', 'Tx.Licenca', '35', '000', '', '001.276/2011', 'Parcelamento:027.121/2023', '']
['2008', 'Tx.Licenca', '35', '000', '', '001.276/2011', 'Parcelamento:027.121/2023', '']
['2009', 'Tx.Licenca', '35', '000', '', '001.276/2011', 'Parcelamento:027.121/2023', '']
['2010', 'Tx.Licenca', '35', '000', '', '001.276/2011', 'Parcelamento:027.121/2023',

In [ ]:
def get_dados_cadastrais_mobiliario(driver, tipo, inscricao):
    url_mobiliario = Config.URL_MOB
    driver.get(url_mobiliario)

    campo_cadastro = get_element_by_xpath(driver, '//input[@name="cadm"]')
    campo_cadastro.clear()
    campo_cadastro.send_keys('inscricao')

    campo_digito = get_element_by_xpath(driver, '//input[@name="digito"]')
    campo_digito.clear()
    campo_digito.send_keys('0')

    campo_documento = get_element_by_xpath(driver, '//input[@name="docto"]')
    campo_documento.clear()
    campo_documento.send_keys('0')

    btn_continuar = get_element_by_xpath(driver, '//input[@type="image"]')
    btn_continuar.click()

    btn_consultar_dados = get_element_by_xpath(driver, '//img[@src="/ps_image/btw.php?cont=Consultar D.A."]')
    if btn_consultar_dados is None:
        return None
    btn_consultar_dados.click()

    btn_dados_cadastrais = get_element_by_xpath(driver, '//input[@value="Dados Cadastrais"]')
    btn_dados_cadastrais.click()

    dado_inscricao = None
    dado_razao_social = None
    dado_endereco = None
    dado_documento = None
    dado_nome_fantasia = None
    dado_estabelecimento = None
    dado_tipo_tributo = None

    table_header = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//td')
    dado_inscricao, dado_razao_social = [d.text.strip() for d in table_header][0].split(' - ')

    table_body = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//following::table[1]//tr')
    dado_endereco, dado_documento = ' '.join([d.text for d in table_body][0].split()[:-2]), [d.text for d in table_body][0].split()[-1]
    dado_endereco += f' - {[d.text for d in table_body][1]}'
    dado_nome_fantasia = [d.text for d in table_body][2].split(':')[-1]
    dado_estabelecimento = [d.text for d in table_body][3].split(':')[-1]
    dado_tipo_tributo = [d.text for d in table_body][4].split(':')[-1]

    table_body = driver.find_elements(By.XPATH, '//input[@value="Dados Cadastrais"]//ancestor::table[@class="titulo"]//following::table[2]//tr')
    socio_list = []
    for i, row in enumerate(table_body):
        dados = row.find_elements(By.TAG_NAME, 'td')
        if i == 0:
            continue
        socio = [d.text for d in dados]
        socio_list.append(socio)

    list_dados = []
    for socio in socio_list:
        dados = [
            dado_inscricao,
            dado_razao_social,
            dado_endereco,
            dado_documento,
            dado_nome_fantasia,
            dado_estabelecimento,
            dado_tipo_tributo,
        ] + socio

        list_dados(tuple(dados))
    
    return list_dados